In [1]:
import os
import pandas as pd
import numpy as np
import sklearn as sk
import warnings
import itertools as IT
import glob
import datetime as dt
import time
import sys

In [2]:
from dateutil.relativedelta import relativedelta

# from xgboost import plot_importance
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import RobustScaler


# #Common Model Algorithms
# from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
# import xgboost

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',500)
pd.set_option('max_info_columns',500)
pd.set_option('display.float_format', lambda x : '%.3f' % x )

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
#from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import NearMiss
#from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold


from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD

In [3]:
data=pd.read_csv("PJT002_train.csv",encoding="UTF-8-sig")

data.shape

print('NON FIRE', round(data['fr_yn'].value_counts()[0]/len(data) * 100,2), '% of the dataset')
print('FIRE', round(data['fr_yn'].value_counts()[1]/len(data) * 100,2), '% of the dataset')

NON FIRE 87.07 % of the dataset
FIRE 12.93 % of the dataset


In [4]:
data

,dt_of_fr,fr_yn,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,prcpttn,wnd_spd,wnd_drctn,hmdt,gas_engry_us_201401,ele_engry_us_201401,gas_engry_us_201402,ele_engry_us_201402,gas_engry_us_201403,ele_engry_us_201403,gas_engry_us_201404,ele_engry_us_201404,gas_engry_us_201405,ele_engry_us_201405,gas_engry_us_201406,ele_engry_us_201406,gas_engry_us_201407,ele_engry_us_201407,gas_engry_us_201408,ele_engry_us_201408,gas_engry_us_201409,ele_engry_us_201409,gas_engry_us_201410,ele_engry_us_201410,gas_engry_us_201411,ele_engry_us_201411,gas_engry_us_201412,ele_engry_us_201412,gas_engry_us_201501,ele_engry_us_201501,gas_engry_us_201502,ele_engry_us_201502,gas_engry_us_201503,ele_engry_us_201503,gas_engry_us_201504,ele_engry_us_201504,gas_engry_us_201505,ele_engry_us_201505,gas_engry_us_201506,ele_engry_us_201506,gas_engry_us_201507,ele_engry_us_201507,gas_engry_us_201508,ele_engry_us_201508,gas_engry_us_201509,ele_engry_us_201509,gas_engry_us_201510,ele_engry_us_201510,gas_engry_us_201511,ele_engry_us_201511,gas_engry_us_201512,ele_engry_us_201512,gas_engry_us_201601,ele_engry_us_201601,gas_engry_us_201602,ele_engry_us_201602,gas_engry_us_201603,ele_engry_us_201603,gas_engry_us_201604,ele_engry_us_201604,gas_engry_us_201605,ele_engry_us_201605,gas_engry_us_201606,ele_engry_us_201606,gas_engry_us_201607,ele_engry_us_201607,gas_engry_us_201608,ele_engry_us_201608,gas_engry_us_201609,ele_engry_us_201609,gas_engry_us_201610,ele_engry_us_201610,gas_engry_us_201611,ele_engry_us_201611,gas_engry_us_201612,ele_engry_us_201612,gas_engry_us_201701,ele_engry_us_201701,gas_engry_us_201702,ele_engry_us_201702,gas_engry_us_201703,ele_engry_us_201703,gas_engry_us_201704,ele_engry_us_201704,gas_engry_us_201705,ele_engry_us_201705,gas_engry_us_201706,ele_engry_us_201706,gas_engry_us_201707,ele_engry_us_201707,gas_engry_us_201708,ele_engry_us_201708,gas_engry_us_201709,ele_engry_us_201709,gas_engry_us_201710,ele_engry_us_201710,gas_engry_us_201711,ele_engry_us_201711,gas_engry_us_201712,ele_engry_us_201712,gas_engry_us_201801,ele_engry_us_201801,gas_engry_us_201802,ele_engry_us_201802,gas_engry_us_201803,ele_engry_us_201803,gas_engry_us_201804,ele_engry_us_201804,gas_engry_us_201805,ele_engry_us_201805,gas_engry_us_201806,ele_engry_us_201806,gas_engry_us_201807,ele_engry_us_201807,gas_engry_us_201808,ele_engry_us_201808,gas_engry_us_201809,ele_engry_us_201809,gas_engry_us_201810,ele_engry_us_201810,gas_engry_us_201811,ele_engry_us_201811,gas_engry_us_201812,ele_engry_us_201812,lw_13101010,lw_13101110,lw_13101210,lw_13101211,lw_13101310,lw_13101410,lw_13111010,lw_13111110,lw_13121010,lw_13121011,lw_13131010,lw_13131110,lw_13141010,lw_13141011,jmk,id,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,trgt_crtr,fr_fghtng_fclt_spcl_css_5_yn,fr_fghtng_fclt_spcl_css_6_yn,us_yn,dngrs_thng_yn,slf_fr_brgd_yn,blk_dngrs_thng_mnfctr_yn,cltrl_hrtg_yn
0,2017-10-20 05:54,Y,단독주택,블록구조,3,69.420,69.420,0.000,1977.000,1.000,0.000,주거용,10.700,nan,0.900,200.000,96.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,na